# Monthly Running Average

In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import xarray            as xr
import os          as os

def geo_idx(dd, dd_array):
    geo_idx = (np.abs(dd_array - dd)).argmin()
    return geo_idx

## File placehodler

In [2]:
display(os.system("date"))



model_member = np.array([1],np.int16)
scenario     = "ssp585"
model        = "GFDL-ESM4"
member       = "r1i1p1f1"

variable       = "tasmax" 
cell_method    = "time: mean over months   time: mean over 30 years "
cell_methodsdv = "time: mean over months   time: stdev over 30 years "


combined_file = "./LOCA2-CONUS-MONTHLY_MEAN___tasmax___GFDL-ESM4.r1i1p1f1___historical-ssp585.nc"
ds = xr.open_dataset(filename_or_obj = combined_file).drop(["time_bnds"])

lon  = ds["lon"]
lat  = ds["lat"]


i_target = geo_idx(360 - 103.2310, lon).values
j_target = geo_idx(       44.0805, lat).values

print(i_target, (360 - 103.2310), lon[i_target].values)
print(j_target,        (44.0805), lat[j_target].values)


display(ds)


display(os.system("date"))


Mon Dec 18 16:32:31 EST 2023


0

356 256.769 256.78125
323 44.0805 44.09375


<xarray.Dataset>
Dimensions:   (time: 1812, lon: 944, bnds: 2, lat: 474)
Coordinates:
  * time      (time) datetime64[ns] 1950-01-16T12:00:00 ... 2100-12-16T12:00:00
  * lon       (lon) float32 234.5 234.6 234.7 234.7 ... 293.3 293.3 293.4 293.5
  * lat       (lat) float32 23.91 23.97 24.03 24.09 ... 53.28 53.34 53.41 53.47
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lon, bnds) float32 ...
    lat_bnds  (lat, bnds) float32 ...
    tasmax    (time, lat, lon) float32 ...
Attributes:
    CDI:               Climate Data Interface version 2.3.0 (https://mpimet.m...
    Conventions:       CF-1.6
    title:             LOCA statistically downscaled climate model data, Davi...
    author:            David W. Pierce, Scripps Institution of Oceanography
    experiment_id:     historical
    model_source_id:   GFDL-ESM4
    model_variant_id:  r1i1p1f1
    history:           Fri Dec 15 23:27:20 2023: cdo --verbose cat LOCA2-CONU...
    NCO:               netCDF Operators version 5.1.1-alpha06 (Homepage = htt...
    frequency:         mon
    CDO:               Climate Data Operators version 2.3.0 (https://mpimet.m...

Mon Dec 18 16:32:34 EST 2023


0

In [3]:
print("-----------------")
display("Enter Block",os.system("date"))

time = ds["time"]


nt = ds["time"].shape[0]
nx = ds[ "lon"].shape[0]
ny = ds[ "lat"].shape[0]
nm = 12
ny = nt/12


start_year = time.dt.year.min().values
end_year   = time.dt.year.max().values
year  = np.arange(start = start_year,
                  stop  = end_year+1,
                  dtype = np.int16)
month = np.arange(start = 1,
                  stop  = 12+1,
                  dtype = np.int16)

yeardv = xr.DataArray(data   =  year,
                    dims   = "year",
                    coords = {"year": year},
                    attrs  = {"description": "calendar year",
                                "long_name": "calendar year"})

monthdv = xr.DataArray(data   =  month,
                      dims    = "month",
                      coords  = {"month": month},
                      attrs   = {"description": "calendar month",
                                   "long_name": "calendar month"})
print("-----------------")
display("Start Reindexing",os.system("date"))

multiindex_ds = ds.assign_coords(month =monthdv,
                                 year  =yeardv
).stack(
    time2d=("year","month")
).reset_index(
    "time", drop=True
).rename(
    time="time2d"
).unstack("time2d")


print("-----------------")
display("End Reindexing",os.system("date"))
print("-----------------")
display("DATASET MULTIINDEX_DS: ",multiindex_ds)
print("-----------------")
display("Start Rollng Mean",os.system("date"))


rolling_monthly = multiindex_ds[variable].rolling(year   =   30,
                                                  center = True).mean().dropna(dim = "year",
                                                                               how =  "all") 
rolling_monthly.expand_dims(dim={"model_member" : 1}) 
rolling_monthly["name"] = variable
rolling_monthly.attrs["cell_methods"] = cell_method
print("-----------------")



display("End Rolling",os.system("date"))
print("-----------------")
display("MONTHLY Rolling: ",rolling_monthly)
display("saving",os.system("date"))
print("-----------------")

outdata.to_netcdf(path           =   "./rolling_monthly_llym.nc", 
                  mode           =           'w', 
                  format         =     "NETCDF4",
                  engine         =    "h5netcdf",
                  encoding       = {variable: {        "dtype": "int16", 
                                                "scale_factor":     0.1,
                                                  "add_offset":     0.0,   
                                                  "_FillValue":  -32767}}) 
display("End All",os.system("date"))
print("-----------------")


-----------------
Mon Dec 18 16:32:34 EST 2023


'Enter Block'

0

-----------------
Mon Dec 18 16:32:34 EST 2023


'Start Reindexing'

...

Mon Dec 18 16:38:32 EST 2023


'saving'

0

-----------------


NameError: name 'outdata' is not defined

In [ ]:
monthly   = multiindex_ds[variable][j_target,i_target,:,:]
r_monthly = outdata[variable][j_target, i_target,:,:]



monthly[  :,5].plot()
r_monthly[:,5].plot()

monthly[  :,2].plot()
r_monthly[:,2].plot()
